In [18]:
import os
from dotenv import load_dotenv
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_groq import ChatGroq
# Load environment variables from .env file
load_dotenv()
llm=ChatGroq(model_name="Gemma2-9b-It", api_key=os.getenv("GROQ_API_KEY"))

In [19]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [20]:
# Now Python can read the key
tavily_api_key = os.getenv("TAVILY_API_KEY")

# Initialize Tavily
tavily_tool = TavilySearchResults(tavily_api_key=tavily_api_key)
tavily_tool.invoke("what is the weather in SF")

[{'title': 'Weather San Francisco in September 2025: Temperature & Climate',
  'url': 'https://en.climate-data.org/north-america/united-states-of-america/california/san-francisco-385/t/september-9/',
  'content': '| 12. September | 17 °C | 62 °F | 23 °C | 73 °F | 13 °C | 56 °F | 14 °C | 57 °F | 0.1 mm | 0.0 inch. |\n| 13. September | 17 °C | 62 °F | 23 °C | 73 °F | 13 °C | 56 °F | 14 °C | 57 °F | 0.1 mm | 0.0 inch. |\n| 14. September | 17 °C | 62 °F | 22 °C | 71 °F | 13 °C | 56 °F | 14 °C | 58 °F | 0.3 mm | 0.0 inch. |\n| 15. September | 16 °C | 61 °F | 21 °C | 70 °F | 13 °C | 55 °F | 14 °C | 58 °F | 0.0 mm | 0.0 inch. | [...] | Max. Temperature °C (°F) | 14 °C (57.3) °F | 14.9 °C (58.7) °F | 16.2 °C (61.2) °F | 17.4 °C (63.3) °F | 19.2 °C (66.5) °F | 21.5 °C (70.8) °F | 21.8 °C (71.2) °F | 22.2 °C (71.9) °F | 23.1 °C (73.6) °F | 21.3 °C (70.3) °F | 17.1 °C (62.8) °F | 13.9 °C (57.1) °F |\n| Precipitation / Rainfall mm (in) | 113 (4) | 118 (4) | 83 (3) | 40 (1) | 21 (0) | 6 (0) | 2 (0)

In [21]:
from langchain.agents import Tool
from langchain_core.prompts import PromptTemplate
from langchain.agents import AgentExecutor , create_self_ask_with_search_agent

In [22]:
template = '''Answer the following questions as best you can. You have access to the following tools:
{tools}
Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question
Begin!
Question: {input}
Thought:{agent_scratchpad}'''

In [23]:
prompt = PromptTemplate.from_template(template)

### Self-Ask with Search Agent:

## Definition:

This approach allows the AI to ask itself follow-up questions when it doesn't immediately know the answer. It performs a recursive questioning process to break down complex queries into smaller, more manageable ones.
Once the AI generates these follow-up questions, it performs an external search (e.g., through Google or an internal database) to gather the necessary information before formulating a response.
How it works:

- Step 1: Receive a complex question.
- Step 2: The agent identifies sub-questions or follow-up questions.
- Step 3: It performs a search or fetches answers to these questions from external resources (like a web search).
- Step 4: The answers are aggregated to provide a complete response.
Key points:

Emphasizes question decomposition.
Relies on external search for sub-questions.
Useful for answering open-ended or broad questions where the answer is not immediately available.
Example:

Original question: “How many moons does Jupiter have?”
Sub-question: “What is Jupiter?” (search)
Sub-question: “What are moons?” (search)
Finally, it retrieves the answer: "Jupiter has 79 moons."


In [27]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    google_api_key=os.getenv("GOOGLE_API_KEY"),
    convert_system_message_to_human=True,
)

In [28]:
from langchain.tools import tool
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent

prompt = hub.pull("hwchase17/self-ask-with-search")
print(prompt.template)

Question: Who lived longer, Muhammad Ali or Alan Turing?
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali

Question: When was the founder of craigslist born?
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952

Question: Who was the maternal grandfather of George Washington?
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washin

In [29]:
from langchain.agents import create_self_ask_with_search_agent
from langchain.agents import AgentExecutor

tools = [
    Tool(
        name="Intermediate Answer",
        func=tavily_tool.run,
        description="useful for when you need to ask with search",
        verbose=True
    )
]


agent = create_self_ask_with_search_agent(llm,tools,prompt)

# make executor with relaxed parsing
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

# now call
response = agent_executor.invoke({"input": "What is the todays gold rate and Pune weather ?"})
print(response)



> Entering new AgentExecutor chain...


/usr/local/lib/python3.12/dist-packages/langchain_google_genai/chat_models.py:499: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Yes.
Follow up: What is the current gold rate?[{'title': 'Gold Price Today in United States', 'url': 'https://goldinforma.com/us', 'content': "goldinforma logo\n\n# Gold Price Today in United States\n\n13th Sep, 2025 | 12:00AM IST\n\n## 24K Gold /g\n\n## 22K Gold /g\n\n## 18K Gold /g\n\nToday's gold rate in United States, dated 13th Sep, 2025, is\nUSD117.18 per gram for 24K gold,\nUSD107.42 per gram for 22K gold, and\nUSD87.89 per gram for 18K gold.\nToday, the prices have seen a \nslight rise\n, with 24K gold up by USD0.51,\nand 22K gold up by USD0.47,\nand 18K gold up by USD0.39 [...] Gold Informa is providing gold price in United States herewith for our readers' informational purposes only.\nThe above gold rates are indicative and do not include taxes. For exact rates, contact your local jeweler.\n\n## Today's Gold Rates by Karat in United States\n\n### Today 24 Carat Gold Price Per Gram in United States (USD)\n\nThe price of 24K gold today, 13th Sep, 2025 in United States has \nris

/usr/local/lib/python3.12/dist-packages/langchain_google_genai/chat_models.py:499: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Intermediate answer: As of September 13, 2025, the price of 24K gold is USD 117.18 per gram.
Follow up: What is the weather in Pune?[{'title': 'Cloudy Skies, Frequent Rainfall Ahead: Pune Weather Forecast ...', 'url': 'https://www.punekarnews.in/cloudy-skies-frequent-rainfall-ahead-pune-weather-forecast-september-13-18/', 'content': 'Skip to content\n\n Home\n Pune\n Cloudy Skies, Frequent Rainfall Ahead: Pune Weather Forecast September 13–18\n\n Pune\n\n# Cloudy Skies, Frequent Rainfall Ahead: Pune Weather Forecast September 13–18\n\nShare this News:\n\nFacebookTwitterWhatsAppLinkedInTelegramMessage\n\nPune, 12th September 2025: The India Meteorological Department (IMD) has forecast a wet spell for Pune and adjoining areas over the next week, with cloudy skies, rain, and thunderstorms expected on most days. [...] ### IMA Urges CM Fadnavis to Withdraw Decision Allowing CCMP Doctors Registration; Warns of Statewide Strike\n\n### Pune: PMC Demolishes 15,000 Sq Ft of Unauthorized Construc

/usr/local/lib/python3.12/dist-packages/langchain_google_genai/chat_models.py:499: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Intermediate answer: On September 13, 2025, Pune is expected to have generally cloudy conditions with very light to light rain.
So the final answer is: As of September 13, 2025, the price of 24K gold is USD 117.18 per gram. The weather forecast for Pune on the same day is generally cloudy with a chance of very light to light rain.

> Finished chain.
{'input': 'What is the todays gold rate and Pune weather ?', 'output': 'As of September 13, 2025, the price of 24K gold is USD 117.18 per gram. The weather forecast for Pune on the same day is generally cloudy with a chance of very light to light rain.'}
